In [1]:
import requests
from bs4 import BeautifulSoup,Comment
import pandas as pd
import time
import os
import sys
import unicodedata
import re
# URL of the NBA awards page
import numpy as np
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.static import players,teams

# Get current season year in the format "2023-24" for example

# URL of the NBA awards page
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup


def pull_bref(ps=False,totals=False):
    leagues = "playoffs" if ps else "leagues"
    frames = []
    for year in range(2025, 2026):
        if totals ==True:
            url = f"https://www.basketball-reference.com/{leagues}/NBA_{year}_totals.html"
            pt_index=28
        else:
            url = f"https://www.basketball-reference.com/{leagues}/NBA_{year}_per_poss.html"
            pt_index=28
        print(url)
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.encoding = 'utf-8' 
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the specific table
        table = soup.find('table')
        
        # Get all rows from the table body
        rows = table.find('tbody').find_all('tr')
        
        # Define the data structure to store extracted rows
        data = []
        for row in rows:
            cells = row.find_all('td')
            if cells:
                # Extract player name, url, team, and stats required
                player_cell = cells[0]
                player_name = player_cell.text if player_cell.text else "N/A"  # Player name
                player_url = "https://www.basketball-reference.com" + player_cell.a['href'] if player_cell.a else "N/A"  # Player URL
                team_acronym = cells[2].text if cells[2].text else "N/A"  # Team acronym
                
                gp = cells[4].text if len(cells) > 4 else "0"  # Minutes played
                mp = cells[6].text if len(cells) > 6 else "0"  # Minutes played

                # Columns required for True Shooting Percentage calculation
                fga = cells[8].text if len(cells) > 8 else "0"  # Field Goal Attempts
                fg = cells[7].text if len(cells) > 7 else "0"   # Field Goals Made
                tpa = cells[11].text if len(cells) > 11 else "0" # Three-Point Attempts
                tp = cells[10].text if len(cells) > 10 else "0"  # Three-Point Made
                fta = cells[18].text if len(cells) > 18 else "0" # Free Throw Attempts
                ft = cells[17].text if len(cells) > 17 else "0"  # Free Throws Made
                pts = cells[pt_index].text if len(cells) > pt_index else "0"  #
                
                data.append([
                    player_name, player_url, team_acronym, year, gp,mp, fga, fg, tpa, tp, fta, ft,pts
                ])
        
        # Create DataFrame for the current year
        year_data = pd.DataFrame(
            data=data, 
            columns=['player', 'url', 'team', 'year', 'G','MP', 'FGA', 'FG', '3PA', '3P', 'FTA', 'FT','PTS']
        )
        frames.append(year_data)
        print(f"Year {year} data added.")
        time.sleep(2)
    
    return pd.concat(frames)


index_frame=pull_bref(ps=False,totals=True)
index_frame['bref_id']=index_frame['url'].str.split('/',expand=True)[5]
index_frame['bref_id']=index_frame['bref_id'].str.split('.',expand=True)[0]
master = pd.read_csv('index_master.csv')
match_dict=dict(zip(master['bref_id'],master['nba_id']))

team_dict=dict(zip(master['team'],master['team_id']))

search_dict={
    "hollaro01": 1641842,
    "sarral01": 1642259,
    "dadiepa01": 1642359,
    "cuiyo01": 1642385,
    "dasiltr01": 1641783,

}
match_dict.update(search_dict)
index_frame['nba_id']=index_frame['bref_id'].map(match_dict)



current_season = "2024-25"  # Update this to the current season if needed

# Fetch all players for the current season
players_data = commonallplayers.CommonAllPlayers(is_only_current_season=1, season=current_season)
players_list = players_data.get_data_frames()[0]

# Display a list of player names
player_names = dict(zip(players_list['DISPLAY_FIRST_LAST'],players_list['PERSON_ID']))

notfound=index_frame[index_frame.year==2025].reset_index(drop=True)
notfound=index_frame[index_frame.nba_id.isna()].reset_index(drop=True)
notfound['nba_id']=notfound['player'].map(player_names)
notfound.dropna(inplace=True)
index_frame.dropna(inplace=True)



index_frame=pd.concat([index_frame,notfound])
index_frame['team_id']=index_frame['team'].map(team_dict)
index_copy = index_frame[['player', 'url', 'year', 'team', 'bref_id', 'nba_id', 'team_id']]
master=master[master.year!=2025]
master=pd.concat([master,index_copy])
master.drop_duplicates(inplace=True)
master.to_csv('index_master.csv',index=False)
index_frame.dropna(subset='bref_id',inplace=True)
index_frame['FTA']=index_frame['FTA'].astype(int)
index_frame['FGA']=index_frame['FGA'].astype(int)

index_frame['PTS']=index_frame['PTS'].astype(int)
year=2025
old_scoring=pd.read_csv('totals.csv')
old_scoring=old_scoring[old_scoring.year<year]
old_scoring.columns

index_frame['TS%'] = (index_frame['PTS'] / (2 * (index_frame['FGA'] + 0.44 * index_frame['FTA']))) * 100

# Select and rename columns to match scoring.csv
new_df = index_frame[['player', 'TS%', 'PTS', 'MP', 'team', 'G', 'FTA','FGA','year', 'nba_id']].copy()
new_df = new_df.rename(columns={
    'player': 'Player',
    'team': 'Tm'
})

# Display the resulting DataFrame
new_scoring=pd.concat([old_scoring,new_df])
new_scoring.fillna(0,inplace=True)
new_scoring.replace([np.inf, -np.inf], 0, inplace=True)
new_scoring.loc[new_scoring['TS%'] > 150, 'TS%'] = 0

new_scoring.to_csv('totals.csv',index=False)
new_scoring[new_scoring.nba_id==2544]

https://www.basketball-reference.com/leagues/NBA_2025_totals.html
Year 2025 data added.


,Player,TS%,PTS,MP,Tm,G,FTA,FGA,year,nba_id
12867,LeBron James,48.807838,1654.0,3122.0,CLE,79,460.0,1492.0,2004,2544.0
13464,LeBron James,55.376202,2175.0,3388.0,CLE,80,636.0,1684.0,2005,2544.0
14036,LeBron James,56.804636,2478.0,3361.0,CLE,79,814.0,1823.0,2006,2544.0
14576,LeBron James,55.249191,2132.0,3190.0,CLE,78,701.0,1621.0,2007,2544.0
15132,LeBron James,56.782621,2250.0,3027.0,CLE,75,771.0,1642.0,2008,2544.0
15724,LeBron James,59.129078,2304.0,3054.0,CLE,81,762.0,1613.0,2009,2544.0
16318,LeBron James,60.435090,2258.0,2966.0,CLE,76,773.0,1528.0,2010,2544.0
16922,LeBron James,59.407222,2111.0,3063.0,MIA,79,663.0,1485.0,2011,2544.0
17505,LeBron James,60.544795,1683.0,2326.0,MIA,62,502.0,1169.0,2012,2544.0
18041,LeBron James,64.049327,2036.0,2877.0,MIA,76,535.0,1354.0,2013,2544.0


In [2]:

def pull_bref_score(ps=False,totals=False):
    leagues = "playoffs" if ps else "leagues"
    frames = []
    for year in range(2025, 2026):
   
        url = f"https://www.basketball-reference.com/{leagues}/NBA_{year}_per_poss.html"

        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        
        response.encoding = 'utf-8' 
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the specific table
        table = soup.find('table')
        
        # Get all rows from the table body
        rows = table.find('tbody').find_all('tr')
        
        # Define the data structure to store extracted rows
        data = []
        for row in rows:
            cells = row.find_all('td')
            if cells:
                pt_index=28
                # Extract player name, url, team, and stats required
                player_cell = cells[0]
                player_name = player_cell.text if player_cell.text else "N/A"  # Player name
                player_url = "https://www.basketball-reference.com" + player_cell.a['href'] if player_cell.a else "N/A"  # Player URL
                team_acronym = cells[2].text if cells[2].text else "N/A"  # Team acronym
                
                gp = cells[4].text if len(cells) > 4 else "0"  # Minutes played
                mp = cells[6].text if len(cells) > 6 else "0"  # Minutes played

                # Columns required for True Shooting Percentage calculation
                fga = cells[8].text if len(cells) > 8 else "0"  # Field Goal Attempts
                fg = cells[7].text if len(cells) > 7 else "0"   # Field Goals Made
                tpa = cells[11].text if len(cells) > 11 else "0" # Three-Point Attempts
                tp = cells[10].text if len(cells) > 10 else "0"  # Three-Point Made
                fta = cells[21].text if len(cells) > 21 else "0" # Free Throw Attempts
                ft = cells[20].text if len(cells) > 20 else "0"  # Free Throws Made
                pts = cells[pt_index].text if len(cells) > pt_index else "0"  # Free Throws Made
                
                data.append([
                    player_name, player_url, team_acronym, year, gp,mp, fga, fg, tpa, tp, fta, ft,pts
                ])
        
        # Create DataFrame for the current year
        year_data = pd.DataFrame(
            data=data, 
            columns=['player', 'url', 'team', 'year', 'G','MP', 'FGA', 'FG', '3PA', '3P', 'FTA', 'FT','PTS']
        )
        frames.append(year_data)
        print(year_data.iloc[0])
        print(f"Year {year} data added.")
        time.sleep(2)
    
    return pd.concat(frames)
index_frame=pull_bref_score(ps=False)
index_frame['bref_id']=index_frame['url'].str.split('/',expand=True)[5]
index_frame['bref_id']=index_frame['bref_id'].str.split('.',expand=True)[0]

match_dict=dict(zip(master['bref_id'],master['nba_id']))

team_dict=dict(zip(master['team'],master['team_id']))

search_dict={
    "hollaro01": 1641842,
    "sarral01": 1642259,
    "dadiepa01": 1642359,
    "cuiyo01": 1642385,
    "dasiltr01": 1641783,

}
match_dict.update(search_dict)
index_frame['nba_id']=index_frame['bref_id'].map(match_dict)


current_season = "2024-25"  # Update this to the current season if needed

# Fetch all players for the current season
players_data = commonallplayers.CommonAllPlayers(is_only_current_season=1, season=current_season)
players_list = players_data.get_data_frames()[0]

# Display a list of player names
player_names = dict(zip(players_list['DISPLAY_FIRST_LAST'],players_list['PERSON_ID']))

notfound=index_frame[index_frame.year==2025].reset_index(drop=True)
notfound=index_frame[index_frame.nba_id.isna()].reset_index(drop=True)
notfound['nba_id']=notfound['player'].map(player_names)
notfound.dropna(inplace=True)
index_frame.dropna(inplace=True)



index_frame=pd.concat([index_frame,notfound])
index_frame['team_id']=index_frame['team'].map(team_dict)

index_frame.dropna(subset='bref_id',inplace=True)
index_frame.fillna(0,inplace=True)
print(index_frame)
index_frame.replace('',0)
index_frame['FTA']=index_frame['FTA'].astype(float)
index_frame['FGA']=index_frame['FGA'].astype(float)

index_frame['PTS']=index_frame['PTS'].astype(float)
year=2025
old_scoring=pd.read_csv('scoring.csv')
old_scoring=old_scoring[old_scoring.year<year]
old_scoring.columns

index_frame['TS%'] = (index_frame['PTS'] / (2 * (index_frame['FGA'] + 0.44 * index_frame['FTA']))) * 100

# Select and rename columns to match scoring.csv
new_df = index_frame[['player', 'TS%', 'PTS', 'MP', 'team', 'G', 'year', 'nba_id']].copy()
new_df = new_df.rename(columns={
    'player': 'Player',
    'team': 'Tm'
})

# Display the resulting DataFrame
new_scoring=pd.concat([old_scoring,new_df])
new_scoring.fillna(0,inplace=True)
new_scoring.loc[new_scoring['TS%'] > 150, 'TS%'] = 0

new_scoring.to_csv('scoring.csv',index=False)
gp=new_scoring[['nba_id','Player','year','G']].reset_index()
gp.to_csv('../player_sheets/lineups/games.csv',index=False)

new_scoring
new_scoring[new_scoring.nba_id==2544]

player                                   Jeenathan Williams
url       https://www.basketball-reference.com/players/w...
team                                                    HOU
year                                                   2025
G                                                         1
MP                                                        2
FGA                                                    24.4
FG                                                     24.4
3PA                                                    24.4
3P                                                     24.4
FTA                                                    48.8
FT                                                      0.0
PTS                                                    73.3
Name: 0, dtype: object
Year 2025 data added.
                 player                                                url  \
0    Jeenathan Williams  https://www.basketball-reference.com/players/w...   
1         James Wis

,Player,TS%,PTS,MP,Tm,G,year,nba_id
12867,LeBron James,48.726159,28.0,3122.0,CLE,79,2004,2544.0
13464,LeBron James,55.322581,34.3,3388.0,CLE,80,2005,2544.0
14036,LeBron James,56.811628,39.4,3361.0,CLE,79,2006,2544.0
14576,LeBron James,55.149356,35.3,3190.0,CLE,78,2007,2544.0
15132,LeBron James,56.759546,39.6,3027.0,CLE,75,2008,2544.0
15724,LeBron James,59.061957,40.8,3054.0,CLE,81,2009,2544.0
16318,LeBron James,60.371891,40.0,2966.0,CLE,76,2010,2544.0
16922,LeBron James,59.446041,36.4,3063.0,MIA,79,2011,2544.0
17505,LeBron James,60.637892,38.1,2326.0,MIA,62,2012,2544.0
18041,LeBron James,64.185951,37.5,2877.0,MIA,76,2013,2544.0


In [3]:

index=pd.read_csv('index_master.csv')
index[index.year==2025]

,player,url,year,team,bref_id,nba_id,team_id
28481,Jayson Tatum,https://www.basketball-reference.com/players/t...,2025,BOS,tatumja01,1628369.0,1610612738
28482,De'Aaron Fox,https://www.basketball-reference.com/players/f...,2025,SAC,foxde01,1628368.0,1610612758
28483,Giannis Antetokounmpo,https://www.basketball-reference.com/players/a...,2025,MIL,antetgi01,203507.0,1610612749
28484,Anthony Davis,https://www.basketball-reference.com/players/d...,2025,LAL,davisan02,203076.0,1610612747
28485,Anthony Edwards,https://www.basketball-reference.com/players/e...,2025,MIN,edwaran01,1630162.0,1610612750
...,...,...,...,...,...,...,...
28940,Mac McClung,https://www.basketball-reference.com/players/m...,2025,ORL,mccluma01,1630644.0,1610612753
28941,Dariq Whitehead,https://www.basketball-reference.com/players/w...,2025,BRK,whiteda01,1641727.0,1610612751
28942,Riley Minix,https://www.basketball-reference.com/players/m...,2025,SAS,minixri01,1642434.0,1610612759
28943,Drew Peterson,https://www.basketball-reference.com/players/p...,2025,BOS,peterdr01,1641809.0,1610612738
